In [ ]:
# Load the packages

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.metrics import mean_squared_error,confusion_matrix, classification_report, roc_curve, auc, accuracy_score, roc_auc_score


# Data Integration and Cleaning
Merging datasets from four different data sources (games, players, plays, and tackles) into a comprehensive dataset, ensuring data quality and consistency.


In [ ]:
# Load the dataset

games_df = pd.read_csv("/Users/ved/Downloads/nfl-big-data-bowl-2024/games.csv")
players_df = pd.read_csv("/Users/ved/Downloads/nfl-big-data-bowl-2024/players.csv")
plays_df = pd.read_csv("/Users/ved/Downloads/nfl-big-data-bowl-2024/plays.csv")
tackles_df = pd.read_csv("/Users/ved/Downloads/nfl-big-data-bowl-2024/tackles.csv")

In [ ]:
#preview the datasets
games_df.head(5), players_df.head(5), plays_df.head(5), tackles_df.head(5)

**Insights from datasets:**
* Players data: include players’ heights, weights, and position may 
* Play data: column ‘down’ and 'yardsToGo’ provide crucial information about the ongoing play.

In [ ]:
tackles_df.describe()

From the above table, we can see that the success tackle rate is around 56.9%.

# Exploratory Data Analysis
To understand the factors that might influence the success tackle rate, we would further perform a more detailed analysis by considering various aspects of the data.

First, I would start with merging the four datasets together using 'nflId', 'gameId', and 'playId'.

In [ ]:
# Merge tackles data with player data
combined_df_1 = pd.merge(tackles_df, players_df, on = 'nflId')

# Then merge with play data
combined_df_2 = pd.merge(combined_df_1, plays_df, on = ['gameId', 'playId'])

# Further merge with games data
data = pd.merge(combined_df_2, games_df, on = 'gameId')

data

In [ ]:
# Convert height to numerical numbers
def height_to_inches(height_str):
    if isinstance(height_str, str):
        parts = height_str.split('-')
        if len(parts) == 2:
            feet, inches = int(parts[0]), int(parts[1])
            return feet * 12 + inches
    return height_str  # if not string, return as is

# Apply the conversion to the 'height' column
data['height'] = data['height'].apply(height_to_inches)

In [ ]:
data.describe()

Investigate the distribution of tackles across various attributes and identify patterns and correlations. Techniques like heatmaps and histograms are used for this analysis.


## 1. Correlation Matrix Heatmap
I started with looking at the distribution of tackles across different player attributes, play characteristics, and game context:
* player attributes (ex: height, weight, position)
* play characteristics (ex: down, yards to go)
* game context (ex: score, quarter, assist, forced Fumble, pff_missedTackle)

In [ ]:
# Selecting relevant features for analysis
features = ['tackle', 'weight', 'height', 'down', 'position', 'yardsToGo', 'yardlineNumber', 'assist', 'forcedFumble', 'pff_missedTackle']
eda_df = data[features]

# Convert categorical variables to dummy variables
dummy_columns = pd.get_dummies(eda_df['position'])
eda_df = pd.concat([eda_df, dummy_columns], axis=1)

# Drop the original 'position' column as it's now represented by dummies
eda_df.drop('position', axis=1, inplace=True)

# Calculate the correlation matrix
correlation_matrix = eda_df.corr()

# Generate a heatmap to visualize the correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm')
plt.title('Correlation Matrix Heatmap')
plt.show()

The heatmap above illustrates the correlation coefficients between different features in our dataset. Darker red squares indicate a stronger positive correlation, while darker blue squares signify a stronger negative correlation. The diagonal line shows a perfect positive correlation (correlation coefficient of 1) where each feature correlates with itself.

Notably, the features `assist` and `forcedFumble` exhibit a positive correlation with `tackle`, suggesting a relationship between these variables and successful tackles. The player positions, represented as dummy variables, also provide interesting insights; for example, `position_CB` correlates with `pff_missedTackle`, which may reflect the involvement of cornerbacks in plays with missed tackles.

Variables with minimal correlation (light-colored squares) will be considered carefully for their relevance in predictive modeling. It's important to remember that correlation does not equal causation, and this heatmap solely reflects linear relationships.

In [ ]:
# Calculate the correlation matrix
correlation_matrix = eda_df.corr()

# Sort the correlations with 'tackle'
sorted_correlation_with_tackle = correlation_matrix['tackle'].sort_values(key = abs, ascending = False)

# Exclude the 'tackle' variable itself to avoid a self-correlation of 1
sorted_correlation_with_tackle = sorted_correlation_with_tackle[sorted_correlation_with_tackle.index != 'tackle']

# Now, visualize this sorted correlation with a bar plot
plt.figure(figsize=(10, 8))
sns.barplot(x=sorted_correlation_with_tackle.values, y=sorted_correlation_with_tackle.index)
plt.title('Correlation with Tackle Success')
plt.xlabel('Correlation Coefficient')
plt.ylabel('Features')
plt.show()

## 2. Distribution of Tackles by Player
Next, I wanted to look at the performance and frequency of tackles by each player.


In [ ]:
tackle_players = data.groupby(['nflId'])['tackle'].agg('sum')

# Creating a DataFrame by passing a dictionary
frame = {'nfl_id': tackle_players.index, 'tackles': tackle_players}
agg_data = pd.DataFrame(frame).reset_index()
agg_data = agg_data.drop(columns=['nfl_id'], axis=1)

# Plotting using matplotlib
plt.figure(figsize=(10, 6))
plt.hist(agg_data['tackles'], bins=20, color='blue', edgecolor='black')
plt.title('Distribution of Tackles by Players')
plt.xlabel('Tackles')
plt.ylabel('Count')
plt.grid(True)
plt.show()

## 3. Distribution of tackles by player height and weight
Then, I started looking at players’ heights and weights to find patterns related to the occurrence of successful tackles.

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(data[data['tackle'] == 1]['height'], kde=True, color='blue', label='Successful Tackles')
sns.histplot(data[data['tackle'] == 0]['height'], kde=True, color='red', label='Unsuccessful Tackles')
plt.title('Distribution of Tackles by Player Height')
plt.legend()

plt.subplot(1, 2, 2)
sns.histplot(data[data['tackle'] == 1]['weight'], kde=True, color='blue', label='Successful Tackles')
sns.histplot(data[data['tackle'] == 0]['weight'], kde=True, color='red', label='Unsuccessful Tackles')
plt.title('Distribution of Tackles by Player Weight')
plt.legend()

plt.tight_layout()
plt.show()

# Feature Engineering

After EDA, I created new features like player BMI and interaction terms to capture complex relationships within the data.


In [ ]:
# Feature 1: Interaction term between 'down' and 'yardsToGo'
data['down_yards_interaction'] = data['down'] * data['yardsToGo']

# Feature 2: Body Mass Index (BMI)
def calculate_bmi(weight, height_in_inches):
    # BMI formula: 703 * weight in pounds / (height in inches)^2
    return 703 * weight / (height_in_inches ** 2)
data['player_bmi'] = data.apply(lambda row: calculate_bmi(row['weight'], row['height']), axis=1)

feature_data = data[['down_yards_interaction', 'player_bmi', 'quarter', 'assist', 'forcedFumble', 'tackle']]
feature_data.head(10)

# Split the data

In [ ]:
# Splitting the dataset into training, validation, and test sets
train, test = train_test_split(feature_data, test_size=0.4, random_state=42)
val, test = train_test_split(test, test_size=0.5, random_state=42)

# Apply get_dummies to each split separately
X_train = pd.get_dummies(train.drop('tackle', axis=1), drop_first=True)
y_train = train['tackle']

X_val = pd.get_dummies(val.drop('tackle', axis=1), drop_first=True)
y_val = val['tackle']

X_test = pd.get_dummies(test.drop('tackle', axis=1), drop_first=True)
y_test = test['tackle']

# Standardize the numerical features
# Note: Only standardize numerical columns. For simplicity, I'm assuming all columns are now either numerical or one-hot encoded
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Now, the dataset should be ready for training and evaluating a model

## 1. Support Vector Machines (SVM):

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X, y):
        self.n_classes = len(np.unique(y))
        self.n_features = X.shape[1]
        self.tree = self._grow_tree(X, y)

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

    def _best_split(self, X, y):
        m = y.size
        if m <= self.min_samples_split:
            return None, None
        num_parent = [np.sum(y == c) for c in range(self.n_classes)]
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None
        for idx in range(self.n_features):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes
            num_right = num_parent.copy()
            for i in range(1, m):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes)
                )
                gini = (i * gini_left + (m - i) * gini_right) / m
                if thresholds[i] == thresholds[i - 1]:
                    continue
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        return best_idx, best_thr

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(
            num_samples=y.size,
            num_samples_per_class=num_samples_per_class,
            predicted_class=predicted_class,
        )
        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node

    def _predict(self, inputs):
        node = self.tree
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

class Node:
    def __init__(self, num_samples, num_samples_per_class, predicted_class):
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

class RandomForest:
    def __init__(self, n_trees=10, max_depth=10, min_samples_split=2, n_samples=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_samples = n_samples
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            if self.n_samples is None:
                X_subset, y_subset = X, y
            else:
                indices = np.random.choice(len(X), size=self.n_samples, replace=True)
                X_subset, y_subset = X[indices], y[indices]
            tree.fit(X_subset, y_subset)
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.zeros((len(X), len(self.trees)))
        for i, tree in enumerate(self.trees):
            predictions[:, i] = tree.predict(X)
        predictions = np.round(predictions).astype(int)  # Round predictions to the nearest integer
        return np.array([np.argmax(np.bincount(predictions[i])) for i in range(len(X))])

# Load and prepare your data (X_train, y_train, X_test, y_test)
feature_data = data[['down_yards_interaction', 'player_bmi', 'quarter', 'assist', 'forcedFumble', 'tackle']]

# Splitting the dataset into training and test sets
train, test = train_test_split(feature_data, test_size=0.4, random_state=42)

# Apply get_dummies to each split separately
X_train = pd.get_dummies(train.drop('tackle', axis=1), drop_first=True)
y_train = train['tackle'].astype(int)  # Convert y_train to integer

X_test = pd.get_dummies(test.drop('tackle', axis=1), drop_first=True)
y_test = test['tackle'].astype(int)  # Convert y_test to integer

# Standardize the numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an instance of the RandomForest classifier
rf_clf = RandomForest(n_trees=100, max_depth=10, min_samples_split=2, n_samples=None)

# Train the model
rf_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_clf.predict(X_test)

# Evaluate the model's performance
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy}")

# Compute the feature importances
feature_importances = np.zeros(X_train.shape[1])
for tree in rf_clf.trees:
    feature_importances += np.zeros(X_train.shape[1])
    node = tree.tree
    while node.left:
        feature_importances[node.feature_index] += node.num_samples
        node = node.left if np.sum(node.left.num_samples) > np.sum(node.right.num_samples) else node.right
feature_importances /= len(rf_clf.trees)

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.bar(range(X_train.shape[1]), feature_importances)
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importances')
plt.show()

## 2. Logistic Regression

In [ ]:
logistic_model = LogisticRegression(fit_intercept=True, max_iter=1000).fit(X_train,y_train)

logistic_pred_test = logistic_model.predict_proba(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy}")

fpr, tpr, threshold = roc_curve(y_test, logistic_pred_test[:,1])
roc_auc = auc(fpr, tpr)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
confusion_matrix(y_test, (logistic_pred_test[:,1] > 0.5))

* Accuracy: 0.878/ AUC: 0.87
* Given our binary outcome (successful or unsuccessful tackle), I adopt logistic regression to predict the probability of a tackle’s success based on our selected input features. An AUC of 0.87 shows a high level of model performance, with the model having a good measure of separability and being able to distinguish between success and failure tackles.

## 3. Random Forest Classifier

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class DecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None

    def fit(self, X, y):
        self.n_classes = len(np.unique(y))
        self.n_features = X.shape[1]
        self.tree = self._grow_tree(X, y)

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

    def _best_split(self, X, y):
        m = y.size
        if m <= self.min_samples_split:
            return None, None
        num_parent = [np.sum(y == c) for c in range(self.n_classes)]
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None
        for idx in range(self.n_features):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes
            num_right = num_parent.copy()
            for i in range(1, m):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes)
                )
                gini = (i * gini_left + (m - i) * gini_right) / m
                if thresholds[i] == thresholds[i - 1]:
                    continue
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        return best_idx, best_thr

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.n_classes)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(
            num_samples=y.size,
            num_samples_per_class=num_samples_per_class,
            predicted_class=predicted_class,
        )
        if depth < self.max_depth:
            idx, thr = self._best_split(X, y)
            if idx is not None:
                indices_left = X[:, idx] < thr
                X_left, y_left = X[indices_left], y[indices_left]
                X_right, y_right = X[~indices_left], y[~indices_left]
                node.feature_index = idx
                node.threshold = thr
                node.left = self._grow_tree(X_left, y_left, depth + 1)
                node.right = self._grow_tree(X_right, y_right, depth + 1)
        return node

    def _predict(self, inputs):
        node = self.tree
        while node.left:
            if inputs[node.feature_index] < node.threshold:
                node = node.left
            else:
                node = node.right
        return node.predicted_class

class Node:
    def __init__(self, num_samples, num_samples_per_class, predicted_class):
        self.num_samples = num_samples
        self.num_samples_per_class = num_samples_per_class
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

class RandomForest:
    def __init__(self, n_trees=10, max_depth=10, min_samples_split=2, n_samples=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_samples = n_samples
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            if self.n_samples is None:
                X_subset, y_subset = X, y
            else:
                indices = np.random.choice(len(X), size=self.n_samples, replace=True)
                X_subset, y_subset = X[indices], y[indices]
            tree.fit(X_subset, y_subset)
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.zeros((len(X), len(self.trees)))
        for i, tree in enumerate(self.trees):
            predictions[:, i] = tree.predict(X)
        predictions = np.round(predictions).astype(int)  # Round predictions to the nearest integer
        return np.array([np.argmax(np.bincount(predictions[i])) for i in range(len(X))])

# Load and prepare your data (X_train, y_train, X_test, y_test)
feature_data = data[['down_yards_interaction', 'player_bmi', 'quarter', 'assist', 'forcedFumble', 'tackle']]

# Splitting the dataset into training and test sets
train, test = train_test_split(feature_data, test_size=0.4, random_state=42)

# Apply get_dummies to each split separately
X_train = pd.get_dummies(train.drop('tackle', axis=1), drop_first=True)
y_train = train['tackle'].astype(int)  # Convert y_train to integer

X_test = pd.get_dummies(test.drop('tackle', axis=1), drop_first=True)
y_test = test['tackle'].astype(int)  # Convert y_test to integer

# Standardize the numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an instance of the RandomForest classifier
rf_clf = RandomForest(n_trees=100, max_depth=10, min_samples_split=2, n_samples=None)

# Train the model
rf_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_clf.predict(X_test)

# Evaluate the model's performance
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy}")

# Compute the feature importances
feature_importances = np.zeros(X_train.shape[1])
for tree in rf_clf.trees:
    feature_importances += np.zeros(X_train.shape[1])
    node = tree.tree
    while node.left:
        feature_importances[node.feature_index] += node.num_samples
        node = node.left if np.sum(node.left.num_samples) > np.sum(node.right.num_samples) else node.right
feature_importances /= len(rf_clf.trees)

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.bar(range(X_train.shape[1]), feature_importances)
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importances')
plt.show()

## 4. Decision Tree

In [ ]:
# function for printing a tree
import pydot
from io import StringIO
from IPython.display import Image
def print_tree(estimator, features, class_names=None, filled=True):
  tree = estimator
  names = features
  color = filled
  classn = class_names
  dot_data = StringIO()
  export_graphviz(estimator, out_file=dot_data,feature_names=features,class_names=classn, filled=filled)
  graph = pydot.graph_from_dot_data(dot_data.getvalue())
  return(graph)

In [ ]:
tackle_tree = DecisionTreeClassifier(min_samples_split=3,min_impurity_decrease=0.000001)
tackle_tree.fit(X_train, y_train)
graph, = print_tree(tackle_tree, features= train.drop(columns = ['tackle']).columns)
Image(graph.create_png())

In [ ]:
tackle_tree_preds = tackle_tree.predict_proba(X_val)

In [ ]:
fpr, tpr, threshold = roc_curve(val['tackle'], tackle_tree_preds[:,1])
roc_auc = auc(fpr, tpr)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

# Result